# Binary Task CNN Notebook

### Contains Code for Binary task CNN Model creation, training and testing<br/> CNN will be an implementation of AlexNet equivalent structure 5 convolutional layers and 3 fully connected layers

In [71]:
# Import required libraries
import numpy as np
from numpy import expand_dims
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

#tqdm is for progress bar functionality in code, must be installed for code to function (TO DO: include exception if tqdm not imported )
from tqdm import tqdm


from sklearn.metrics import roc_curve, roc_auc_score

import pickle as pkl
import cv2

#Libraries for CNN model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

# 1. Loading Labels and Resized Image Dataset

In [27]:
#Reading created pkl files for binary labels and image data.
Binary_labels = pd.read_pickle('./dataset/Y_Binary_label.pkl')
MRI_2D_imgs = pd.read_pickle('./dataset/CNN_Images_2D_DF.pkl')

In [28]:
MRI_2D_img_array = np.array(MRI_2D_imgs)
MRI_img_array = MRI_2D_img_array.reshape(MRI_2D_img_array.shape[0],50,50)

ERROR! Session/line number was not unique in database. History logging moved to new session 193


In [69]:
print(MRI_img_array.shape)

(3000, 50, 50)
ERROR! Session/line number was not unique in database. History logging moved to new session 212


In [85]:
#Adding 4th channel to array (for convnet fitting)
#The last channel is
MRI_img_array_channel = MRI_img_array.reshape(3000,50,50,1)

In [86]:
print(MRI_img_array_channel.shape)

(3000, 50, 50, 1)


In [87]:
#Getting our Y and X inputs for the model and scaling the X inputs
Y = Binary_labels[['MRI_Binary_Label']]

X = MRI_img_array_channel/255

In [88]:
# Split the data into training and testing(70% training and 30% testing data)
xTrain,xTest,yTrain,yTest=train_test_split(X, Y, train_size = 0.7)

# 2. CNN Model Building

In [89]:
#Initialising the Model
Binary_CNN = Sequential(name = 'Binary_CNN')

#We use a smaller number of filters compared to original Alexnet implementation
#Testing computational power first
#Original Filter numbers/4 so:
#24 64 96 96 64

#1st Convolutional Layer
#Input arguements
#Filters: Number of output filters in convolution
Binary_CNN.add(Conv2D(filters = 24, input_shape = (MRI_img_array.shape[1], MRI_img_array.shape[2], 1), kernel_size = (11, 11), strides = (4, 4), padding = 'same', name="conv0"))
Binary_CNN.add(BatchNormalization(name = 'bn0'))
Binary_CNN.add(Activation('relu'))
Binary_CNN.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))


#2nd Convolutional Layer
Binary_CNN.add(Conv2D(filters = 64, kernel_size = (5, 5), strides = (1, 1), padding = 'same', name="conv1"))
Binary_CNN.add(BatchNormalization(name = 'bn1'))
Binary_CNN.add(Activation('relu'))
Binary_CNN.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))


#3rd Convolutional Layer
Binary_CNN.add(Conv2D(filters = 96, kernel_size = (3, 3), strides = (1, 1), padding = 'same', name="conv2"))
Binary_CNN.add(BatchNormalization(name = 'bn2'))
Binary_CNN.add(Activation('relu'))


#4th Convolutional Layer
Binary_CNN.add(Conv2D(filters = 96, kernel_size = (3, 3), strides = (1, 1), padding = 'same', name="conv3"))
Binary_CNN.add(BatchNormalization(name = 'bn3'))
Binary_CNN.add(Activation('relu'))


#5th Convolutional Layer
Binary_CNN.add(Conv2D(filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', name="conv4"))
Binary_CNN.add(BatchNormalization(name = 'bn4'))
Binary_CNN.add(Activation('relu'))
Binary_CNN.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))


#Passing through the Fully connected layers
#We set the number of neurons per layer as 2500 for now
#Less than orignal 4096
Binary_CNN.add(Flatten())

#1st Fully Connected Layer
Binary_CNN.add(Dense(2500, name = 'fc0'))
Binary_CNN.add(BatchNormalization(name = 'bnfc0'))
Binary_CNN.add(Activation('relu'))
Binary_CNN.add(Dropout(0.4))

#2nd Fully Connected Layer
Binary_CNN.add(Dense(2500, name = 'fc1'))
Binary_CNN.add(BatchNormalization(name = 'bnfc1'))
Binary_CNN.add(Activation('relu'))
Binary_CNN.add(Dropout(0.4))

#3rd Fully Connected Layer
Binary_CNN.add(Dense(500, name = 'fc2'))
Binary_CNN.add(BatchNormalization(name = 'bnfc2'))
Binary_CNN.add(Activation('relu'))
Binary_CNN.add(Dropout(0.4))


#Output Layer
Binary_CNN.add(Dense(1, name = 'fcOut'))
Binary_CNN.add(BatchNormalization())
Binary_CNN.add(Activation('sigmoid'))



Binary_CNN.summary()

Model: "Binary_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 13, 13, 24)        2928      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 13, 13, 24)        96        
_________________________________________________________________
activation_65 (Activation)   (None, 13, 13, 24)        0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 7, 7, 24)          0         
_________________________________________________________________
conv1 (Conv2D)               (None, 7, 7, 64)          38464     
_________________________________________________________________
bn1 (BatchNormalization)     (None, 7, 7, 64)          256       
_________________________________________________________________
activation_66 (Activation)   (None, 7, 7, 64)          0

In [90]:
#Compiling the model
Binary_CNN.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

In [ ]:
def Train_Model():
    

In [45]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

ERROR! Session/line number was not unique in database. History logging moved to new session 203


In [ ]:
#Setting up training and testing data generators for image inputs
xTrain_imggen = ImageDataGenerator(rescale = 1./255)
xTest_imggen = ImageDataGenerator(rescale = 1./255)

#Setting up generator from our image array xTrain and xTest
xTrain_generator = xTrain_imggen.flow(xTrain, batch_size = 50)


#from tensorflow.keras.callbacks import TensorBoard
#tensorboard = TensorBoard(log_dir="./logs/Binary_CNN")

In [91]:
#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr = ReduceLROnPlateau(   monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5)


In [92]:
#Defining model training parameters
batch_size = 50
epochs = 10
learn_rate = 0.001


In [93]:
#Training the Model
History = Binary_CNN.fit(xTrain, yTrain, batch_size= batch_size, epochs= epochs, verbose = 2, callbacks = [lrr])


Epoch 1/10
 - 19s - loss: 0.6121 - accuracy: 0.7767


D:\Programs\Anaconda\envs\gym\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/10
 - 2s - loss: 0.5072 - accuracy: 0.8838
Epoch 3/10
 - 2s - loss: 0.4510 - accuracy: 0.9424
Epoch 4/10
 - 2s - loss: 0.4284 - accuracy: 0.9552
Epoch 5/10
 - 2s - loss: 0.3990 - accuracy: 0.9633
Epoch 6/10
 - 2s - loss: 0.3796 - accuracy: 0.9724
Epoch 7/10
 - 2s - loss: 0.3610 - accuracy: 0.9781
Epoch 8/10
 - 2s - loss: 0.3387 - accuracy: 0.9867
Epoch 9/10
 - 2s - loss: 0.3214 - accuracy: 0.9871
Epoch 10/10
 - 2s - loss: 0.3120 - accuracy: 0.9876


In [94]:
print(History.history.keys())

dict_keys(['loss', 'accuracy', 'lr'])
ERROR! Session/line number was not unique in database. History logging moved to new session 214


In [95]:
acc = History.history['accuracy'][-1]
loss = History.history['loss'][-1]


print("Training accuracy: ", acc)
print("Training loss: ", loss)


Training accuracy:  0.98761904
Training loss:  0.3120093721719015


In [96]:
num_epochs = range(1, len(History.history['accuracy'])+1)
